In [1]:
%reload_ext autoreload
%autoreload 2
import logging
import grpc
import pandas as pd
import sys
sys.path.append('proto')
from proto import contest_pb2
from proto import contest_pb2_grpc
from proto import question_pb2
from proto import question_pb2_grpc
import numpy as np
import time
from utils import *
from strategy_utils import *

In [2]:
contest_channel=grpc.insecure_channel('47.103.23.116: 56702')
question_channel=grpc.insecure_channel('47.103.23.116: 56701')

contest_stub=contest_pb2_grpc.ContestStub(contest_channel)
question_stub=question_pb2_grpc.QuestionStub(question_channel)

login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

session_key=login_response.session_key
init_capital=login_response.init_capital

In [7]:
i=0#控制seq
count=0
data_lst=[]
factors_lst=[]
period=1#eg 每两天跑一次策略
comission=0#
max_exposure=0.1#大盘上涨，多头增加，大盘下跌，空头增加
single_stock_position_limit=0.1#
lending_rate=0.01#
borrow_rate=0.05#
leverage=2#
factors=pd.DataFrame()
prev_factors=pd.DataFrame()#上一次计算的factors

In [8]:
while True:
    login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

    session_key=login_response.session_key
    init_capital=login_response.init_capital
    i = 0
    try:
        while True:
            question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88,sequence=i))
            print(question_response.sequence)
            if question_response.sequence!=-1:
                dailystk = [x.values for x in question_response.dailystk]
                data_lst.extend(dailystk)

                if count>10:#开始不动，只要有新数据就跑一次策略
                    print('run strategy')
                    df=pd.DataFrame(data_lst,columns=['day','stock','open','high','low','close','volume'],
                                    dtype=float).set_index(['day','stock'])
                    
                    dailyfactors=get_factors(df,prev_factors)  #从数据获取因子
                    prev_factors=dailyfactors

                    factors_lst.extend(dailyfactors.values)#向因子库追加
                    #部份因子可能需要根据历史因子数据归一化

                    # factor_select=select_factors(factors,n=10,period=period)  #计算相关系数选取因子
                    # factor_select=['avg', 'mom', 'max52', 'cci', 'K', 'D', 'J', 'rsi', 'trix', 'willr']
                    # factor_select=['avg','mom', 'vol', 'max52', 'K', 'D', 'J', 'rsi', 'trix', 'willr', 'macd', 'natr','mfi']
                    factor_select=['avg', 'mom', 'max52', 'D', 'willr', 'natr', 'rsi']

                    # index_direction='neutral'#TODO 大盘方向，用于控制exposure

                    weights = get_weight(dailyfactors[factor_select],head_n=10,tail_n=10)
                    if count < 100:
                        target_pos=get_position(weights,
                                                pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                                question_response.positions,
                                                question_response.capital / 2,
                                                comission)
                    else:
                        target_pos=get_position(weights,
                                                pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                                question_response.positions,
                                                question_response.capital,
                                                comission)
                    
                    #提交策略
                    if count%period==0:#按周期提交
                        ##summit answer
                        submit_response = contest_stub.submit_answer(contest_pb2.AnswerRequest(user_id=88,user_pin='dDTSvdwk',session_key=login_response.session_key,sequence=i,positions=target_pos))

                        print(submit_response,question_response.capital)
                        if not submit_response.accepted:
                            print(submit_response.reason)
                            if submit_response.reason[-7:] == 'timeout':
                                i=question_response.sequence+1
                                count+=1
                                continue#如果提交超时，直接请求新数据
                
                i=question_response.sequence+1
                count+=1
            time.sleep(1)
            # if count==20:
                # break
    except Exception as e:
        print(e)
    time.sleep(1)


run strategy
stock
1001.0     256713
1002.0     226328
1003.0    -396843
1006.0    -119745
1008.0     681948
           ...   
1327.0    2160172
1333.0      28702
1334.0      -6492
1346.0     512532
1349.0      51092
Name: close, Length: 107, dtype: int32
accepted: true
 630165588.6678315
-1
-1
-1
589
run strategy
stock
1001.0     261304
1002.0     236477
1003.0    -209218
1005.0     -80835
1006.0    -123271
           ...   
1317.0     523480
1327.0    2232489
1334.0      -6611
1346.0     532851
1349.0     106765
Name: close, Length: 102, dtype: int32
accepted: true
 631222914.4771137
-1
-1
-1
590
run strategy
stock
1000.0    -158360
1001.0     163321
1004.0     -34571
1006.0    -114664
1008.0     674276
           ...   
1327.0    2137556
1334.0      -6183
1346.0     494190
1347.0     146049
1349.0      50526
Name: close, Length: 105, dtype: int32
accepted: true
 623850046.2937163
-1
-1
-1
591
run strategy
stock
1001.0    156719
1012.0    -26823
1013.0    156464
1022.0    -24009
102

KeyboardInterrupt: 